In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Provided text (augmented with some additional similar sentences)
data = """hello world this is a simple text dataset for RNN example. 
We need more data for better results. 
hello world this is an example for text sequence prediction. 
RNN and LSTM can help in predicting the next word in a sequence. 
this is a simple LSTM example with text data. 
We need more data for better sequence prediction results."""

# Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
total_words = len(tokenizer.word_index) + 1  # +1 for padding

# Convert text to sequences
input_sequences = [] 
token_list = tokenizer.texts_to_sequences([data])[0]
for i in range(1, len(token_list)):
    for j in range(1, len(token_list[i:])+1):
        n_gram_sequence = token_list[i-1:i+j]
        input_sequences.append(n_gram_sequence)

# Pad sequences
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# Create predictors and label
X, y = input_sequences[:,:-1], input_sequences[:,-1]
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

model = Sequential()
model.add(Embedding(total_words, 10, input_length=max_sequence_len-1))
model.add(LSTM(100))
model.add(Dense(total_words, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# model.summary()

history = model.fit(X, y, epochs=200, verbose=1)

def predict_next_word(model, tokenizer, text, max_sequence_len):
    token_list = tokenizer.texts_to_sequences([text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)
    return tokenizer.index_word[predicted[0]]

seed_text = "RNN"
next_words = 7

for _ in range(next_words):
    next_word = predict_next_word(model, tokenizer, seed_text, max_sequence_len)
    seed_text += " " + next_word

print(seed_text)

Epoch 1/200
54/54 ━━━━━━━━━━━━━━━━━━━━ 7s 48ms/step - accuracy: 0.0559 - loss: 3.3945
Epoch 2/200
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 43ms/step - accuracy: 0.0741 - loss: 3.2452
Epoch 3/200
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - accuracy: 0.2033 - loss: 2.8205
Epoch 4/200
54/54 ━━━━━━━━━━━━━━━━━━━━ 3s 52ms/step - accuracy: 0.3726 - loss: 2.0216
Epoch 5/200
54/54 ━━━━━━━━━━━━━━━━━━━━ 6s 64ms/step - accuracy: 0.5850 - loss: 1.4982
Epoch 6/200
54/54 ━━━━━━━━━━━━━━━━━━━━ 5s 58ms/step - accuracy: 0.7569 - loss: 1.1369
Epoch 7/200
54/54 ━━━━━━━━━━━━━━━━━━━━ 5s 88ms/step - accuracy: 0.8419 - loss: 0.8921
Epoch 8/200
54/54 ━━━━━━━━━━━━━━━━━━━━ 3s 45ms/step - accuracy: 0.8977 - loss: 0.6991
Epoch 9/200
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 44ms/step - accuracy: 0.9082 - loss: 0.5654
Epoch 10/200
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - accuracy: 0.9284 - loss: 0.4383
Epoch 11/200
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 43ms/step - accuracy: 0.9376 - loss: 0.3801
Epoch 12/200
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step